In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 19 10:10:23 2019

"""

import pandas as pd
from sys2_query_15 import sys2Query_15
from sys1_query import sys1Query
from queries_PIM import sys2_basic_query, sys1_attr_query, sys1_name_query, sys1_basic_query


gcom = sys1Query()
sys2 = sys2Query_15()

In [ ]:
def sys2_skus(sys1_skus):
    """get basic list of sys2 SKUs to pull the related PIM nodes"""
    sku_list = sys1_skus['sys1_SKU'].tolist()
    sys2_skus = ", ".join("'" + str(i) + "'" for i in sku_list)
    sys2_sku_list = sys2.sys2_q15(sys2_basic_query, 'tprod."supplierSku"', sys2_skus)
    
    return sys2_sku_list


def sys2_atts(query, node, query_type):
    """pull sys2 attributes based on the PIM node list created by sys2_skus"""
    df = pd.DataFrame()
    #pull attributes for the next pim node in the sys2 list
    df = sys2.sys2_q15(query, query_type, node)
    print('sys2 ', node)

    return df


def sys1_nodes(node, search_level):
    """basic pull of all nodes if L2 or L3 is chosen"""
    df = pd.DataFrame()
    #pull basic details of all SKUs -- used for gathering L3s if user chooses L2 or L1
    df = gcom.sys1_q(sys1_basic_query, search_level, node)
    
    return df

    
def sys1_atts(node):
    """pull sys1 attributes based on Categiry ID"""
    df = pd.DataFrame()
    #pull attributes for the next pim node in the sys2 list
    df = gcom.sys1_q(sys1_attr_query, 'cat.CATEGORY_ID', node)

    return df


def sys1_by_name(att):
    """pull sys2 attributes based on the PIM node list created by sys2_skus"""
    df = pd.DataFrame()

    if isinstance(att, int):#k.isdigit() == True:
        pass
    else:
        att = "'" + str(att) + "'"
    df = gcom.sys1_q(sys1_name_query, 'attr.DESCRIPTOR_NAME', att)

    if df.empty == True:
        print('sys1_BY_NAME with {} = No results returned', att)
        
    return df


def sys1_values(df):
    """find the top 5 most used values for each attribute and return as sample_values"""
    top_vals = pd.DataFrame()
    temp_att = pd.DataFrame()
    all_vals = pd.DataFrame()
    
    df['Count'] =1
    atts = df['sys1_Attribute_Name'].unique()
    
    vals = pd.DataFrame(df.groupby(['sys1_Attribute_Name', 'sys1_Attribute_Value'])['Count'].sum())
    vals = vals.reset_index()

    for attribute in atts:
        temp_att = vals.loc[vals['sys1_Attribute_Name']== attribute]
        #pull the top 10 values and put into 'sample' field
        temp_att = temp_att.sort_values(by=['Count'], ascending=[False]).head(10)
        top_vals = pd.concat([top_vals, temp_att], axis=0)
        #put all attribute values into a single string for TF-IDF processing later
        temp_df = df.loc[df['sys1_Attribute_Name']== attribute]
        temp_df['sys1 ALL Values'] = ' '.join(item for item in temp_df['sys1_Attribute_Value'] if item)
        all_vals= pd.concat([all_vals, temp_df], axis=0)

    top_vals = top_vals.groupby('sys1_Attribute_Name')['sys1_Attribute_Value'].apply('; '.join).reset_index()
    
    #vals = vals.drop(['Count'], axis=1)
    #vals = vals.groupby('sys1_Attribute_Name')['sys1_Attribute_Value'].apply('; '.join).reset_index()
    all_vals = all_vals.drop_duplicates(subset='sys1_Attr_ID')
    all_vals = all_vals[['sys1_Attr_ID', 'sys1 ALL Values']]
        
    return all_vals, top_vals


def sys2_values(query, node, query_type):
    """find the top 5 most used values for each attribute and return as sample_values"""
    top_vals = pd.DataFrame()
    temp_att = pd.DataFrame()
    all_vals = pd.DataFrame()
    
    df = sys2.sys2_q15(query, query_type, node)
        
    if df.empty==False:
        df['Count'] = 1
        atts = df['sys2_Attribute_Name'].unique()
    
        vals = pd.DataFrame(df.groupby(['sys2_Attribute_Name', 'Normalized Value'])['Count'].sum())
        vals = vals.reset_index()
 
        for attribute in atts:
            temp_att = vals.loc[vals['sys2_Attribute_Name']== attribute]
            #pull the top 10 values and put into 'sample' field
            temp_att = temp_att.sort_values(by=['Count'], ascending=[False]).head(10)
            top_vals = pd.concat([top_vals, temp_att], axis=0)
            #put all attribute values into a single string for TF-IDF processing later            
            temp_df = df.loc[df['sys2_Attribute_Name']== attribute]
            temp_df['sys2 ALL Values'] = ' '.join(item for item in temp_df['Normalized Value'] if item)
            all_vals= pd.concat([all_vals, temp_df], axis=0)
                        
        top_vals = top_vals.groupby('sys2_Attribute_Name')['Normalized Value'].apply('; '.join).reset_index()
        
        all_vals = all_vals.drop_duplicates(subset='sys2_Attr_ID')
        all_vals = all_vals[['sys2_Attr_ID', 'sys2 ALL Values']]
    else:
        print('sys2 node {} NO VALUES'.format(node))
        
    return all_vals, top_vals